## Economic Dispatch (ED)

System operators must select the most cost effective combination of generators, each with distinct cost structures and engineering limitations. ED is a key operational model for the optimization of the variable (short-run) production costs of a fleet of generators, subject to various technical constraints, to meet the total electricity demand. 

- **Characteristics**:
    - Objective: Minimize short-run costs of different producing generators while meeting the total system damand, considering relevant constraints.
    - Constraints: Power balance (total generation = total demand), generation limits (minimum and maximum capacity), and sometimes ramp rates for generators.
    - Assumptions: Assumes the system can adjust generation output in real-time to minimize cost while meeting demand, without considering transmission constraints.Physics of electricity flows and constraints related to turning on or "committing" large thermal generators aswell as network representation are neglected  

- **Advantages**: 
    - Simple and efficient for real-time dispatch decisions.
- **Disadvantages**:
    - Ignores power system constraints like voltage, line flow, and losses.
    - Cannot handle reactive power optimization or voltage stability concerns.

- **Field of Application**:
    - Real-time and short-term dispatch in power markets.
    - Used in situations where the main concern is balancing generation and demand economically without consideration of network constraints.

>[!NOTE]
>- fix costs will not change and would be constant in the objective function -> optimal decision variables would not change by adjusting this constant, therefore, they can safely be ignored for the purposes of finding optimal dispatch, but will have to be considered to calculate producer profits.

### Different complexity levels:  
1. [Single-time period, simple generator constraints](#1-single-time-period-simple-generator-constraints)
2. [Multiple-time period, simple generator constraints](#2-multiple-time-period-simple-generator-constraints)
3. Multiple-time period, complex generator constraints with time coupling (_including ramp up and ramp down_)

### 1. Single-time period, simple generator constraints

**Problem Formulation**: Minimize the electricity production costs to meet the given, static system demand.

$$G = \text{set of generators}$$

Objective Function:  
$$min \sum_{g \in G} 
VarCost_g \times x_g$$

subject to:
$$\begin{aligned}
\sum_g x_g&=Demand \\
x_g &\le P^{max}_g  \quad  \forall g \in G \\
x_g &\ge P^{min}_g   \quad \forall g \in G \\
\end{aligned}$$

Decision Variable:  
$$x_g: \text{generation in MW, produced by each generator g}$$

Parameters:

- $P^{min}_g$, the minimum operating bounds for the generator (based on engineering or natural resource constraints)
- $P^{max}_g$, the maximum operating bounds for the generator (based on engineering or natural resource constraints)
- $Demand$ in MW
- Variable Costs:

$$
VarCost_g = VarOM_g + HeatRate_g \times FuelCost_g
$$
with OM: operating and maintenance. For renewables it reduces to $VarCost_g = VarOM_g$

In [ ]:
import pandas as pd
df_generators=pd.read_csv('generators.csv', sep=',')
df_generators.fillna(0, inplace=True)

df_variables = df_generators.loc[df_generators.is_variable==1]
df_non_variables = df_generators.loc[df_generators.is_variable!=1]

df_generators

#### Pyomo Solution
[_**Single-time period, simple generator constraints**_](#1-single-time-period-simple-generator-constraints)

In [ ]:
import pyomo.environ as pyo

n_generators = len(df_generators.index)
const_total_demand = 400

model = pyo.ConcreteModel()

# Sets
model.G = pyo.RangeSet(n_generators)
model.G_var = pyo.RangeSet(len(df_variables.index))
model.G_non_var = pyo.RangeSet(len(df_non_variables.index))

# Decision Variable
def bounds_rule(model, g):
    return (df_generators.Pmin[g-1], df_generators.Pmax[g-1])
model.x = pyo.Var(
    model.G, 
    within=pyo.NonNegativeReals, 
    bounds=bounds_rule,
)

# Params (here defined over full generator set)
model.VarOM = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.VarOM)))
model.HeatRate = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.HeatRate)))
model.FuelCost = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.FuelCost)))
model.CF = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.CF)))
model.Pmax = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.Pmax)))

# Objective 
model.total_costs=pyo.Objective(
    expr=sum(
        model.x[g] * (model.VarOM[g] + model.HeatRate[g] * model.FuelCost[g]) 
        for g in model.G
    ),
    sense=pyo.minimize
)

# Constraints
def demand_rule(model):
    return sum(model.x[g] for g in model.G) == const_total_demand
model.demand_constraint = pyo.Constraint(rule=demand_rule)

def capacity_rule_var(model, g):
    return model.x[g] == model.CF[g] * model.Pmax[g]
model.capacity_constraint_var = pyo.Constraint(model.G_var, rule=capacity_rule_var)

optimizer = pyo.SolverFactory("appsi_highs")
result = optimizer.solve(model)

result.write()
print(f"total costs: {pyo.value(model.total_costs)}")
# model.x.display()
pd.Series(model.x.extract_values()).plot(grid=True, kind="bar", title="Pyomo Solution", xlabel="#-generator", ylabel="MW")


#### Linopy Solution
[**_Single-time period, simple generator constraints_**](#1-single-time-period-simple-generator-constraints)

In [ ]:
import linopy 

model = linopy.Model()

const_total_demand = 400

# Decision Variable
x = model.add_variables(
    lower=df_generators.Pmin, 
    upper=df_generators.Pmax,
    coords=[df_generators.index],
    name="generators"
)

# no direct need to define Parameters and Sets with linopy

# Constraints
con1 = model.add_constraints(x.sum() == const_total_demand)
def var_demand_rule(model, i):
    return x[i] == df_variables.CF[i] * df_variables.Pmax[i]
con2 = model.add_constraints(var_demand_rule ,coords=[df_variables.index])

# Objective
obj = (x * (df_generators.VarOM + df_generators.HeatRate * df_generators.FuelCost)).sum()
model.add_objective(obj, overwrite=True, sense='min')

model.solve()


In [ ]:
solution = model.solution.to_dataframe()
solution.plot(grid=True, kind="bar", title="Linopy Solution", xlabel="#-generator", ylabel="MW")

### 2. Multiple-time period, simple generator constraints
  
an additional time index is introduced:
      
$$\begin{align}
    \min \sum_{g \in G, t \in T} VarCost_g \times GEN_{g,t} \\
    \text{s.t.}
    \sum_{g} GEN_{g,t} = Demand_t \quad \forall  t \in T \\
    GEN_{g,t} \leq Pmax_{g,t} \quad \forall g \in G , t \in T \\
    GEN_{g,t} \geq Pmin_{g,t} \quad \forall g \in G , t \in T \\
    \end{align}
$$
    
- for conventional resources Pmin and Pmax are constant over time
- for variable (renewable) generators, $Pmax_{g,t}$ varies with time (i.e., based on solar irradiation or wind speeds). $Pmin_{g,t}$ for wind and solar resources is usually 0.
- for hydropower resources minimum streamflow constraints can produce a time-variant parameter for $Pmin_{g,t}$.